# Chain of Thought

This is a recent technique which can improve your LLM's reasoning ability (at the expense of some extra time!)

First lets make sure our libraries are up to date:

In [ ]:
!pip install -U transformers
!pip install git+https://github.com/guidance-ai/guidance

  Cloning https://github.com/guidance-ai/guidance to /tmp/pip-req-build-3ifw11m0
  Running command git clone --filter=blob:none --quiet https://github.com/guidance-ai/guidance /tmp/pip-req-build-3ifw11m0
  Resolved https://github.com/guidance-ai/guidance to commit d36601b62096311988fbba1ba15ae4126fb695df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB/s e

We can now load a chat model using huggingface as before:

In [ ]:
from transformers import pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
pipe = pipeline("conversational", model_name, device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Here's a complex medical question for the model to solve:
> Indented block



In [ ]:
MAIN_PROMPT = """\
A 12-month-old girl is brought in by her mother to the pediatrician for the
first time since her 6-month checkup. The mother states that her daughter
had been doing fine, but the parents are now concerned that their daughter
is still not able to stand up or speak. On exam, the patient has a temperature of 98.5°F (36.9°C), pulse is 96/min, respirations are 20/min, and blood
pressure is 100/80 mmHg. The child appears to have difficulty supporting
herself while sitting. The patient has no other abnormal physical findings.
She plays by herself and is making babbling noises but does not respond
to her own name. She appears to have some purposeless motions. A previous clinic note documents typical development at her 6-month visit and
mentioned that the patient was sitting unsupported at that time. Which of
the following is the most likely diagnosis?

A) Language disorder
B) Rett syndrome
C) Fragile X syndrome
D) Trisomy 21
"""

Lets try just asking this question to the model:

In [ ]:
messages = [
     {"role": "system", "content": "You are a smart and intelligent medical expert."},
     {"role": "user", "content": MAIN_PROMPT},
]

output = pipe(messages)
messages[-1]["content"]

"The most likely diagnosis based on the given information is Fragile X syndrome. Fragile X syndrome is a genetic disorder that affects the development of the brain and can cause intellectual disability, autism, and other developmental delays. The patient's symptoms, including difficulty supporting herself while sitting, babbling noises, and purposeless motions, all suggest that she may have Fragile X syndrome. The previous clinic note from her 6-month visit also mentions that she was sitting unsupported at that time, which is consistent with the symptoms of Fragile X syndrome."

The answer should be B. This is a hard problem so lets turn this into a chain of thought prompt by asking the model to show its working:

In [ ]:
messages = [
     {"role": "system", "content": "You are a smart and intelligent medical expert."},
     {"role": "user", "content": MAIN_PROMPT + "\nLet’s use step by step inductive reasoning, given the medical nature of the question"},
]

output = pipe(messages)
messages[-1]["content"]

"Based on the given information, the most likely diagnosis is Rett syndrome.\n\n1. Language disorder: The patient has difficulty supporting herself while sitting, which is a common symptom of Rett syndrome.\n\n2. Rett syndrome: The patient's previous clinic note documents typical development at her 6-month visit, which is consistent with the symptoms of Rett syndrome.\n\n3. Fragile X syndrome: The patient's mother states that her daughter had been doing fine, but the parents are now concerned that their daughter is still not able to stand up or speak. This is consistent with the symptoms of fragile X syndrome, which can cause difficulty with speech and language development.\n\n4. Trisomy 21: The patient's temperature is 98.5°F (36.9°C), which is consistent with the symptoms of trisomy 21, which can cause developmental delays and intellectual disability.\n\n5. Other: The patient has no other abnormal physical findings, which is not consistent with any of the other diagnoses.\n\nIn concl

# Guiding the model

We can combine this idea with the guidance library to extract the final answer.

In [ ]:
from guidance import models, select, gen
import guidance

lm = models.Transformers(pipe.model, pipe.tokenizer)

We can use the select function to force the model to output (A,B,C,D) after its reasoning:

In [ ]:
@guidance(stateless=False)
def cot(lm):
  messages = [
      {"role": "system", "content": "You are a smart and intelligent medical expert."},
      {"role": "user", "content": MAIN_PROMPT},
  ]
  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  prompt = prompt.replace("</s>", "")

  lm += prompt +  \
    "\nLet’s use step by step inductive reasoning, given the medical nature of the question. " +\
    gen(max_tokens=200)

  return lm

lm += cot()

lm + """<|user|> Therefore, among A-D, which answer is most likely?
<|assistant|> """ + select(["A", "B", "C", "D"])

# Exercises

1. Experiment with different methods for improving prompts. The [Prompt Engineering Guide](https://www.promptingguide.ai) is a good place to start.